In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import  DataLoader
from torchsummary import summary
from celeb import CelebDataset, CelebDatasetNew


In [2]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# hyper params
batch_size = 5 
num_epochs = 20 
learning_rate = 3e-4 #better is 3e-4



In [4]:
# dataset initialization
# transform = transforms.Compose([transforms.PILToTensor(),transforms.Lambda(lambda x: x / 255.0), transforms.Resize([178,178], antialias=True)])
transform = transforms.Compose([transforms.ToTensor(),transforms.Resize([178,178], antialias=True)])

train_dataset = CelebDatasetNew(
    split='train', transform=transform)

test_dataset = CelebDatasetNew(
    split='test', transform=transform)

val_dataset = CelebDatasetNew(
    split='val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size, True, num_workers=7)
test_loader = DataLoader(test_dataset, batch_size, False, num_workers=7)
val_loader = DataLoader(val_dataset, batch_size, False, num_workers=7)

In [5]:
# check dataset
examples = iter(train_loader)
samples = next(examples)
inp, target = samples
print(inp.shape)
print(inp[0])
print(target.shape)

for k in range(0, 6, 2):
    i = inp[k].permute((1, 2, 0))
    plt.subplot(6, 2, k+1)
    plt.imshow(i)
    o = target[k].permute((1, 2, 0))
    plt.subplot(6, 2, k+2)
    plt.imshow(o)
plt.show()

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/subedikushal/.local/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/subedikushal/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/subedikushal/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/subedikushal/project/major/celeb.py", line 138, in __getitem__
    inp_image = io.imread(inp_name)
  File "/home/subedikushal/.local/lib/python3.10/site-packages/skimage/io/_io.py", line 53, in imread
    img = call_plugin('imread', fname, plugin=plugin, **plugin_args)
  File "/home/subedikushal/.local/lib/python3.10/site-packages/skimage/i

In [ ]:
import torchvision.transforms.functional as TF


class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels), 
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels), 
            nn.ReLU(inplace=True),

            
        )

    def forward(self, x):
        return self.conv(x)
    
class UNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super().__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride = 2)

        # down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature
        
        for feature in reversed(features):
            self.ups.append(
                # output = s * (n-1) + k- 2*p
                nn.ConvTranspose2d(
                    feature*2, feature,kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2,feature))
        
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, 1)
    
    def forward(self, x):
        skip_connections = []
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)

        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]
            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])
            concat_skip = torch.cat((skip_connection, x), dim = 1)
            x = self.ups[idx+1](concat_skip)
        
        return self.final_conv(x)

In [ ]:
model = UNET(3,3).to(device)
#summary(model,(3,178,178))

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        inputs = F.sigmoid(inputs)
        targets = F.sigmoid(targets)

        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

In [ ]:
# loss and optimizer
criterion = nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
torch.backends.cudnn.benchmark = True
model.train();

In [ ]:
train_losses= {}
val_losses = {}
for epoch in range(1, num_epochs+1):
    model.train()
    loss = 0
    for input_images,target_images in train_loader:
        input_images = input_images.to(device)
        target_images = target_images.to(device)
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = model(input_images)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, target_images)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    train_losses[epoch] = loss
    model.eval()

    with torch.no_grad():
        loss = 0
        for input_images,target_images in val_loader:
            input_images = input_images.to(device)
            target_images = target_images.to(device)
            
            
            outputs = model(input_images)
            
            # compute training reconstruction loss
            val_loss = criterion(outputs, target_images)
            
            
            
            # add the mini-batch training loss to epoch loss
            loss += val_loss.item()
        loss = loss / len(val_loader)
        val_losses[epoch] = loss

    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch, num_epochs, loss))

# Evaluation Metrics

In [ ]:
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now()

# Format the current date as a string in the "YYYY-MM-DD" format
formatted_date = current_datetime.strftime("%d-%m-%Y(%H-%M)")

import json
with open(f'trainlosses_{formatted_date}.json', 'w') as fp:
    json.dump(train_losses, fp)

with open(f'vallosses_{formatted_date}.json', 'w') as fp:
    json.dump(val_losses, fp)

torch.save(model.state_dict(), f'{formatted_date}.pth')
print("Model Saved.")

In [ ]:
# model.load_state_dict(torch.load("./unet_model_saved/05-12-2023(18-43)-10e-b5-mse.pth"))
model.eval();

In [ ]:
with torch.no_grad():
    h,w = 178,178
    examples = iter(test_loader)
    samples, label = next(examples)

    samples = samples.to(device)
    
    outputs = model(samples)


    print(outputs.shape)
    print(outputs)
    rows = 3
    cols = 3
    for i in range(1,rows*cols,cols):
        one_label = label[i%batch_size].reshape(3,h,h).permute(1,2,0)
        one_label = one_label.cpu().numpy()
        one_output = outputs[i%batch_size].reshape(3,h,h).permute(1,2,0)
        one_output = one_output.cpu().numpy()
        one_sample = samples[i%batch_size].reshape(3,h,h).permute(1,2,0)
        one_sample = one_sample.cpu().numpy()

        print(one_output.shape)
        # input
        plt.subplot(rows,cols,i)
        if i == 1:
            plt.title("Input")
        plt.imshow(one_sample)
        #output
        plt.subplot(rows,cols,i+1)
        if i == 1:
            plt.title("Output")
        plt.imshow(one_output)
        # ground truth
        plt.subplot(rows,cols,i+2)

        if i == 1:
            plt.title("Ground Truth")
        plt.imshow(one_label)
    plt.subplots_adjust(left=0.1,
                        bottom=0.05,
                        right=0.9,
                        top=0.9,
                        wspace=0.4,
                        hspace=0.4)
    plt.show()